In [3]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-07 23:59:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.74MB/s    in 0.2s    

2022-03-07 23:59:51 (4.74 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
#spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
#get initial data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep="\t")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
#check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
#drop null and duplicate values
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

9002021
9001052
9001052


In [7]:
#establish review df
rev2_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
rev2_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10048FCFWVJOH|   34974850|B002OL2MU4|     471222439| 2011-04-07|
|R1008YTRJEH6U7|   30903818|B00LAHPLTW|     355012266| 2014-12-15|
|R100F3WWLNCX5S|   51491202|B003T91K94|     816619821| 2014-07-31|
|R100IJSLJ25VPG|    2572903|B00ENI0JM4|     653653620| 2014-11-30|
|R100JPDEL3NLYL|   37174514|B00EF1OGOG|     558284639| 2014-07-01|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [8]:
# Change review date to date format
from pyspark.sql.types import DateType
rev2_df = rev2_df.withColumn("review_date", rev2_df["review_date"].cast(DateType()))

In [9]:
#establish products df
prod2_df = df.select(["product_id", "product_title"])
prod2_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002OL2MU4|Garmin Portable f...|
|B00LAHPLTW|iPhone 6 case, Ca...|
|B003T91K94|BW 3.6mm Wide Ang...|
|B00ENI0JM4|ePartSolution-OEM...|
|B00EF1OGOG|Anker 2nd Generat...|
+----------+--------------------+
only showing top 5 rows



In [10]:
#check products data types
prod2_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [11]:
#count products
count1 = prod2_df.count()
count1 

9001052

In [12]:
#count distinct products
prod2_df = prod2_df.distinct()

count2 = prod2_df.count()
count2

In [13]:
#establish Customer Df grouped by customer
cust2_df = df.groupBy("customer_id").count()

cust2_df = cust2_df.withColumnRenamed("count", "customer_count")

cust2_df.orderBy(cust2_df["customer_count"].desc()).show(10)


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30208851|           616|
|   53037408|           493|
|   45070473|           437|
|   49266466|           435|
|   32038204|           427|
|   52870270|           353|
|   44834233|           330|
|   11995502|           319|
|   51346302|           318|
|   43856165|           316|
+-----------+--------------+
only showing top 10 rows



In [14]:
#check customer data types 
cust2_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [15]:
#change customer count data type
from pyspark.sql.types import IntegerType

cust2_df = cust2_df.withColumn("customer_count", cust2_df["customer_count"].cast(IntegerType()))
#verify change
cust2_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [8]:
#establish vine df
vine2_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine2_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10048FCFWVJOH|          5|            0|          0|   N|
|R1008YTRJEH6U7|          1|            1|          2|   N|
|R100F3WWLNCX5S|          5|            0|          0|   N|
|R100IJSLJ25VPG|          5|            0|          0|   N|
|R100JPDEL3NLYL|          3|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [13]:
#convert star rating from string to integer
from pyspark.sql.types import IntegerType
vine2_df = vine2_df.withColumn("star_rating", vine2_df["star_rating"].cast(IntegerType()))

In [15]:
#verify change
vine2_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [14]:
#Code to RDS
mode = "append"
jdbc_url="jdbc:postgresql://<MYRDSENDPOINTWASHERE>"
config = {"user":"postgres", "password":"<MYPASSWORDWASHERE>", "driver" :"org.postgresql.Driver"}

In [ ]:
#write to RDS datatable
rev2_df.write.jdbc(url=jdbc_url, table="review_id_table2", mode=mode, properties=config)

In [ ]:
#write to RDS datatable
prod2_df.write.jdbc(url=jdbc_url, table="products2", mode=mode, properties=config)

In [ ]:
#write to RDS datatable
cust2_df.write.jdbc(url=jdbc_url, table="customers2", mode=mode, properties=config)

In [16]:
#write to RDS datatable
vine2_df.write.jdbc(url=jdbc_url, table="vine_table2", mode=mode, properties=config)